In [1]:
pip install numpy scipy cupy-cuda118


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cupy-cuda118 (from versions: none)
ERROR: No matching distribution found for cupy-cuda118


In [1]:
import os
import sys
import glob
import numpy as np
import cupy as cp
from scipy.io import wavfile

def process_folder(folder):
    wav_files = glob.glob(os.path.join(folder, '*.wav'))
    if not wav_files:
        print(f'No WAV files found in {folder}')
        return

    for wav_file in wav_files:
        try:
            # 1. Load audio
            sr, data = wavfile.read(wav_file)

            # 2. To mono
            if data.ndim == 2:
                data = data.mean(axis=1)
            data = data.astype(np.float32)

            # 3. GPU FFT
            gpu_data      = cp.asarray(data)
            N             = gpu_data.shape[0]
            gpu_fft       = cp.fft.fft(gpu_data)
            gpu_fft_half  = gpu_fft[: N//2]
            gpu_freqs     = cp.fft.fftfreq(N, d=1.0/sr)[: N//2]
            gpu_mag       = cp.abs(gpu_fft_half)

            # 4. Bring back to CPU
            freqs     = cp.asnumpy(gpu_freqs)
            magnitude = cp.asnumpy(gpu_mag)

            # 5. Save CSV
            out_csv = os.path.splitext(wav_file)[0] + '.csv'
            header  = 'frequency_hz,magnitude'
            np.savetxt(
                out_csv,
                np.column_stack((freqs, magnitude)),
                delimiter=',',
                header=header,
                comments=''
            )

            print(f'Processed {wav_file} → {out_csv}')

        except Exception as e:
            print(f'Error processing {wav_file}: {e}')

if __name__ == '__main__':
    folder = sys.argv[1] if len(sys.argv) > 1 else '.'
    process_folder(folder)


ModuleNotFoundError: No module named 'cupy'

In [3]:
pip install cupy-cuda118


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cupy-cuda118 (from versions: none)
ERROR: No matching distribution found for cupy-cuda118


In [2]:
pip install cupy

^C
Note: you may need to restart the kernel to use updated packages.
